#### Creating Milestone Dataframe

In [0]:
# Find dataframes where the required field is not null
df = (
    spark.sql("SELECT * FROM data")
    .filter(
        (col("payload_issue.milestone").isNotNull()) 
        | (col("payload_pull_request.milestone").isNotNull())
    )
)

In [0]:
# Columns to access correct information with correct labels
cols = [
    "id",
    col("closed_at").cast(TimestampType()),
    "closed_issues",
    col("created_at").cast(TimestampType()),
    col("creator.id").alias("creator_id"),
    "description",
    col("due_on").cast(TimestampType()),
    "html_url",
    "labels_url",
    "node_id",
    "number",
    "open_issues",
    "state",
    "title",
    col("updated_at").cast(TimestampType()),
    "url",
]

In [0]:
issue_milestone = df.filter(
    col("payload_issue.milestone")
    .isNotNull()
).select("payload_issue.milestone.*")

pull_request_milestone = df.filter(
    col("payload_pull_request.milestone")
    .isNotNull()
).select("payload_pull_request.milestone.*")

In [0]:
# Union of two DFs
milestoneDF = (
    issue_milestone.union(pull_request_milestone)
    .distinct()
    .select(cols)
    .orderBy("id")
)

#### Write to database

In [0]:
milestoneDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/milestone")